In [2]:
# Code resource: https://github.com/Gananath/DrugAI/blob/d1f37917ae75e36b93b7ae4ff3cfc62032e78d68/DrugAI-GAN.py#L43

'''
original Author: Gananath R
DrugAI-GAN: Experiments with GAN for drug like molecule generation
Contact: https://github.com/gananath
'''

'\noriginal Author: Gananath R\nDrugAI-GAN: Experiments with GAN for drug like molecule generation\nContact: https://github.com/gananath\n'

In [3]:

'''
. GAN訓練目標是訓練兩組參數: 1.讓分類準確率最高的判別器參數。  2.欺騙率最高的生成器參數。
. Cost fuction V(G,D) 包含了生成器和判別器的參數。
'''

'\n. GAN訓練目標是訓練兩組參數: 1.讓分類準確率最高的判別器參數。  2.欺騙率最高的生成器參數。\n. Cost fuction V(G,D) 包含了生成器和判別器的參數。\n'

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os.path
import h5py

In [5]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.callbacks import History
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [6]:
# Create folder
if not os.path.exists(os.getcwd()+'\\Output'):
    os.makedirs(os.getcwd()+'\\Output')
# os.path.exists: Return True if path refers to an existing path
# getcwd() returns current working directory of a process.


# seed for random number reproductio
np.random.seed(180221)

In [7]:
os.getcwd()

'C:\\Users\\Eric\\Anaconda3\\4.Algorithm_Practice\\TensorFlow'

# Read data

In [8]:
##read csv file
data = pd.read_csv('stahl.csv')
data = data.reindex(np.random.permutation(data.index))
print('Number of data:',len(data))
data.head(10)

Number of data: 335


,SMILES,cox2,estrogen,gelatinase,neuramidase,kinase,thrombin,none
65,C[N]1C(=CC(=C1C(=O)C2=CC=C(Cl)C=C2)C)CC(O)=O,1,0,0,0,0,0,0
74,CN1C(=C(O)C2=C(C=CC=C2)[S]1(=O)=O)C(=O)NC3=NC=...,1,0,0,0,0,0,0
129,CCC(C1=CC=C(O)C=C1)=C(CC)C2=CC=C(O)C=C2,0,1,0,0,0,0,0
168,CCC1=C([N](N=C1C2=CC=C(O)C=C2)C3=CC=CC=C3)C4=C...,0,1,0,0,0,0,0
99,CC1=CC(=CN=C1)C2=C(C=C(Cl)C=N2)C3=CC=C(C=C3)[S...,1,0,0,0,0,0,0
147,CC12CCC3=C4C=CC(=CC4=CC=C3C1CCC2=O)O,0,1,0,0,0,0,0
3,C[S](=O)(=O)NC1=C(SC2=C(F)C=C(F)C=C2)C=C(C=C1)...,1,0,0,0,0,0,0
269,NC(N)=[NH+]CCCCNC(=O)C1CCCN1C(=O)C([NH3+])CC2=...,0,0,0,0,0,1,0
160,OC1=CC=C(C=C1)C2=C(CC3=CC=C(OCC[NH]4CCCCC4)C=C...,0,1,0,0,0,0,0
92,CCC1=C(Cl)C=C2C=C(C(OC2=C1)C(F)(F)F)C(O)=O,1,0,0,0,0,0,0


In [9]:
# Take the SMILES string
Y=data.SMILES
print(type(Y))
Y.head(2)

<class 'pandas.core.series.Series'>


65         C[N]1C(=CC(=C1C(=O)C2=CC=C(Cl)C=C2)C)CC(O)=O
74    CN1C(=C(O)C2=C(C=CC=C2)[S]1(=O)=O)C(=O)NC3=NC=...
Name: SMILES, dtype: object

In [10]:
X = data.iloc[:,1:7] # in this case, take the data of column 1:7
# DataFrame.iloc: Purely integer-location based indexing for selection by position.
X = X.values
X = X.astype('int')
type(X)

numpy.ndarray

# Data preprocessing

In [51]:
def dimX(x,ts):
    '''Adding timestep to feature'''
    x=np.asarray(x)
    newX=[]
    for i, c in enumerate(x):
        newX.append([])
        for j in range(ts):
            newX[i].append(c)
    return np.array(newX)
    # np.asarray(a): convert the input a to an array

    
def dimY(Y,ts):
    '''adding timestep to target'''
    temp = np.zeros((len(Y), ts, len(chars)), dtype=np.bool) 
    for i, c in enumerate(Y):
        for j, s in enumerate(c):
            #print i, j, s
            temp[i, j, char_idx[s]] = 1
    return np.array(temp)

In [52]:
##padding smiles to same length by adding "|" at the end of smiles
maxY = Y.str.len().max() # find the max length of SMILES 
y = Y.str.ljust(maxY, fillchar='|') # the maxY-th number frim left side, fill '|' 
ts = y.str.len().max()

print(maxY, ts)
y[0]
#  pandas.series.str
#  ljust(): returns the string left justified in a string of length width

105 105


'C[S](=O)(=O)NC1=C(OC2CCCCC2)C=C(C=C1)[N](=O)=O|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||'

In [53]:
# CharToIndex and IndexToChar functions
chars = sorted(list( set("".join(y.values.flatten())))) # list with all chars 
# join() returns a string in which the string elements of 
#        sequence have been joined by str separator.
print('total chars:', len(chars))

char_idx= dict((c, i) for i, c in enumerate(chars))
idx_char = dict((i, c) for i, c in enumerate(chars))

y_dash = dimY(y,ts)
x_dash = dimX(X,ts)

print('y_dash shape:', y_dash.shape)
print('x_dash shape:', x_dash.shape)

total chars: 25
y_dash shape: (335, 105, 25)
x_dash shape: (335, 105, 6)


# Building Model

In [15]:
def Gen():
    '''Generator model: generating sequence with random noise'''
    G = Sequential() # The keras.model.Sequential() model is a linear stack of layers
    G.add(TimeDistributed(Dense(x_dash.shape[2]), input_shape=(x_dash.shape[1], x_dash.shape[2])))
    # intput a onehot encode (x_dash)
    # TimeDistributedDense applies a same Dense (fully-connected) operation to every timestep of a 3D tensor.
    
    # Hidden layer 1 with 216 LSTM cell 
    G.add(LSTM(216, return_sequences=True)) 
    G.add(Dropout(0.3))
    # True: output at all steps. False: output as last step.
    
    # Hidden layer 1 with 216 LSTM cell
    G.add(LSTM(216, return_sequences=True))
    G.add(Dropout(0.3))
    
     # Hidden layer 3 with 216 LSTM cell
    G.add(LSTM(216, return_sequences=True))
    #G.add(BatchNormalization(momentum=0.9))
    
    # output a SMILES sequence(generating) 
    G.add(TimeDistributed(Dense(y_dash.shape[2], activation='softmax')))
    G.compile(loss='categorical_crossentropy', optimizer=Adam(lr=2e-4))
    return G



def Dis():
    '''Discriminator model'''
    D = Sequential()
    D.add(TimeDistributed(Dense(y_dash.shape[2]), input_shape=(y_dash.shape[1], y_dash.shape[2])))
    # Input SMILES sequence
    
    # Hidden layer 1 with 216 LSTM cell
    D.add(LSTM(216, return_sequences=True))
    D.add(Dropout(0.3))
    
    # Hidden layer 1 with 60 LSTM cell
    D.add(LSTM(60, return_sequences=True))
    D.add(Flatten())
    
    # output layer with one neural(Real or Fake)
    D.add(Dense(1, activation='sigmoid'))
    D.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.001))
    return D


def Gan():
    '''Generative adversarial network'''
    GAN=Sequential()
    GAN.add(G) # Generating mosel
    D.trainable=False # Weight of dis model will not change when training GAN
    # How model.trainable = False works in keras (GAN model)
    # https://gist.github.com/naotokui/a9274f12af9d946e99b6df73a5d2af6d
    
    GAN.add(D) # Discriminator model
    GAN.compile(loss='binary_crossentropy', optimizer=Adam(lr=2e-4))
    # Cost function including two model result
    return GAN

In [ ]:
y_dash.shape[2]

In [16]:
#initializing models
G=Gen()
D=Dis()
GAN=Gan()

In [17]:
'''
if os.path.exists(os.getcwd()+"/output/Gen.h5")==True and os.path.exists(os.getcwd()+"/output/Dis.h5")==True and os.path.exists(os.getcwd()+"/output/Gan.h5")==True:
        #loading weights if exits
        G.load_weights(os.getcwd()+"/output/Gen.h5")
        D.load_weights(os.getcwd()+"/output/Dis.h5")
        GAN.load_weights(os.getcwd()+"/output/Gan.h5")
'''     

'\nif os.path.exists(os.getcwd()+"/output/Gen.h5")==True and os.path.exists(os.getcwd()+"/output/Dis.h5")==True and os.path.exists(os.getcwd()+"/output/Gan.h5")==True:\n        #loading weights if exits\n        G.load_weights(os.getcwd()+"/output/Gen.h5")\n        D.load_weights(os.getcwd()+"/output/Dis.h5")\n        GAN.load_weights(os.getcwd()+"/output/Gan.h5")\n'

In [18]:
print("GAN input  : "+str(GAN.input_shape))
print("GAN output : "+str(GAN.output_shape))
print('-----------------------------------')
print("Gen input  : "+str(G.input_shape))
print("Gen output : "+str(G.output_shape))
print('-----------------------------------')
print("Dis input  : "+str(D.input_shape))
print("Dis output : "+str(D.output_shape))

GAN input  : (None, 105, 6)
GAN output : (None, 1)
-----------------------------------
Gen input  : (None, 105, 6)
Gen output : (None, 105, 25)
-----------------------------------
Dis input  : (None, 105, 25)
Dis output : (None, 1)


In [19]:
D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_3 (TimeDist (None, 105, 25)           650       
_________________________________________________________________
lstm_4 (LSTM)                (None, 105, 216)          209088    
_________________________________________________________________
dropout_3 (Dropout)          (None, 105, 216)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 105, 60)           66480     
_________________________________________________________________
flatten_1 (Flatten)          (None, 6300)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 6301      
Total params: 282,519
Trainable params: 0
Non-trainable params: 282,519
_________________________________________________________________


In [20]:
G.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 105, 6)            42        
_________________________________________________________________
lstm_1 (LSTM)                (None, 105, 216)          192672    
_________________________________________________________________
dropout_1 (Dropout)          (None, 105, 216)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 105, 216)          374112    
_________________________________________________________________
dropout_2 (Dropout)          (None, 105, 216)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 105, 216)          374112    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 105, 25)           5425      
Total para

In [21]:
GAN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 105, 25)           946363    
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 282519    
Total params: 1,228,882
Trainable params: 946,363
Non-trainable params: 282,519
_________________________________________________________________


# Pre-training discrimator

In [22]:
def shuffle3D(arr):
    for a in arr:
        np.random.shuffle(a)
        

def trainDis(data=None,mc=None):
    if data is None and mc is None:
        # Train on fake data        
        fake_data= G.predict(x_dash) # Generating fake data
        targets = np.zeros(x_dash.shape[0]).astype(int) # all target = 0
        Dloss = D.fit(fake_data, targets, nb_epoch=1) # Train fake data
             
    elif data is None and mc=="mc":
        #preventing mode collapse
        #artificial noice training 
        fake_ydata = np.copy(y_dash)
        shuffle3D(fake_ydata) # call shuffle3D function to adding noise
        targets = np.zeros(x_dash.shape[0]).astype(int) # all target = 0
        Dloss = D.fit(fake_ydata, targets,nb_epoch=1) # Train shuffling data  
            
    else:
        # Train on real data
        targets = np.ones(x_dash.shape[0]).astype(int) # all target = 1
        Dloss = D.fit(data,targets,nb_epoch=1)  # Train real data   
    return Dloss.history['loss'][0]

In [23]:
D.trainable=True

#pre training
for i in range(5): # con tring 20 or more times
    shuffleData = np.random.permutation(y_dash)
    # np.random.permutation: Randomly permute a sequence, or return a permuted range.
    trainDis() # Train on fake data 
    dloss = trainDis(shuffleData) # Train on shuffle data
    print("Pre Training Discrimator "+str(dloss)+"\n")

C:\Users\Eric\Anaconda3\envs\Tensorflow\lib\site-packages\keras\models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
335/335 [==============================] - 13s 40ms/step - loss: 0.6842
Epoch 1/1
335/335 [==============================] - 9s 28ms/step - loss: 0.6771
Pre Training Discrimator 0.677102665047

Epoch 1/1
335/335 [==============================] - 9s 27ms/step - loss: 0.6839
Epoch 1/1
335/335 [==============================] - 10s 29ms/step - loss: 0.6693
Pre Training Discrimator 0.669257113649

Epoch 1/1
335/335 [==============================] - 10s 29ms/step - loss: 0.6835
Epoch 1/1
335/335 [==============================] - 9s 28ms/step - loss: 0.6623
Pre Training Discrimator 0.66226115458

Epoch 1/1
335/335 [==============================] - 10s 29ms/step - loss: 0.6830
Epoch 1/1
335/335 [==============================] - 10s 29ms/step - loss: 0.6552
Pre Training Discrimator 0.65522992095

Epoch 1/1
335/335 [==============================] - 9s 28ms/step - loss: 0.6825
Epoch 1/1
335/335 [==============================] - 9s 28ms/step - loss: 0.6480
Pre Training Discrimato

# Train GAN Model

In [24]:
def prediction(preds):
    '''prediction of sequence with argmax'''
    y_pred=[]
    for i,c in enumerate(preds):
        y_pred.append([])
        for j in c:
            y_pred[i].append(np.argmax(j))
    return np.array(y_pred)


def seq_txt(y_pred):
    '''sequence to text conversion'''
    newY=[]
    for i,c in enumerate(y_pred):
        newY.append([])
        for j in c:
            newY[i].append(idx_char[j])
    return np.array(newY)


def smiles_output(s):
    '''joined smiles output'''
    smiles=np.array([])
    for i in s:
        j=''.join(str(k) for k in i)
        smiles=np.append(smiles,j)
    return smiles

In [25]:
def trainGAN():
    '''train GAN model'''    
    target = np.ones(x_dash.shape[0]).astype(int) # target = 1
    gan_loss = GAN.fit(x_dash, target,nb_epoch=1)
    
    return gan_loss.history['loss'][0]

In [26]:
episodes = 1

for episode in range(episodes):
    print("Epoch "+str(episode)+"/"+str(episodes))
    trainDis() # Train on fake data 
    
    shuffleData = np.random.permutation(y_dash)
    disloss = trainDis(y_dash) # Train on real data (undate D weight)
    disloss = trainDis(mc="mc") # Train on shuffle (fake) data (undate D weight)    
    
    ganloss = trainGAN() # update G weight   
    print("D loss="+str(disloss)+" GAN loss="+str(ganloss))
    
    if episode%(episodes/10)==0:
        #Saving files            
        G.save_weights(".\\Output\\Gen_mc.h5")
        D.save_weights(".\\Output\\Dis_mc.h5")
        GAN.save_weights(".\\Output\\GAN_mc.h5")
        
    if episode%(episodes/25)==0:
        print("Predicting Molecule")
        x_pred = [[0,0,0,1,0,0],[0,1,0,0,0,0],[0,0,0,0,0,1]]
        x_pred = dimX(x_pred,ts)   
        preds = G.predict(x_pred) # Generating SEMIS
        y_pred = prediction(preds)
        y_pred = seq_txt(y_pred)
        s = smiles_output(y_pred)
        print(s)

Epoch 0/1


C:\Users\Eric\Anaconda3\envs\Tensorflow\lib\site-packages\keras\models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
335/335 [==============================] - 8s 25ms/step - loss: 0.6817
Epoch 1/1
335/335 [==============================] - 9s 27ms/step - loss: 0.6401
Epoch 1/1
335/335 [==============================] - 9s 28ms/step - loss: 0.7200


C:\Users\Eric\Anaconda3\envs\Tensorflow\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/1
335/335 [==============================] - 51s 151ms/step - loss: 0.7115
D loss=0.719996391304 GAN loss=0.711496711845
Predicting Molecule
[ '222222222222222222222222222||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||'
 '|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||'
 '|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||']


# Predict

In [ ]:
##For Prediction

'''
#start Prediction
Ghash=Gen()
Ghash.load_weights('Gen_mc.h5')
x_pred=[[0,0,0,1,0,0],
        [0,1,0,0,0,0],
        [0,0,0,0,0,1]]
x_pred=[[0.6,0,0,0,0,0],
        [.3,0,0,0,0,0],
        [0.7,0,0,0,0,0]]
	
x_pred=dimX(x_pred,ts)      
preds=Ghash.predict(x_pred)
y_pred=prediction(preds)
y_pred=seq_txt(y_pred)
s=smiles_output(y_pred)
print s
#end prediction
'''